# titanic

In [1]:

import numpy as np
import pandas as pd

from keras.models import Sequential
from keras.models import Model

from keras.layers import Input
from keras.layers import Dense
from keras.layers import Activation
from keras.layers import BatchNormalization
from keras.layers import Dropout
from keras.layers import initializers
from keras.layers import regularizers

from keras.optimizers import Adam, Adagrad, Adadelta

import keras.backend as K
from keras.utils.np_utils import to_categorical

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler 
from sklearn.feature_selection import chi2, SelectKBest

import statsmodels.formula.api as smf
import statsmodels.api as sm
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
%matplotlib inline

Using TensorFlow backend.
/Users/jacobanderson/anaconda/envs/deep-learning/lib/python3.5/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


Keras does get its source of randomness from the NumPy random number generator, so this must be seeded regardless of whether you are using a Theano or TensorFlow backend.

In [2]:
from numpy.random import seed
seed(123)
from tensorflow import set_random_seed
set_random_seed(234)


#### Import data

In [4]:
df = pd.read_csv("../processed_data/4_16.csv")
# consider using some sort of select file option - similar to R

In [5]:
df.shape

(1309, 26)

In [120]:
test = pd.read_csv("../raw_data/test (1).csv") #band aid

In [121]:
test.columns

Index(['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch',
       'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [122]:
Pass = pd.DataFrame(test.PassengerId)

In [123]:
Pass.shape #hold on to this guy

(418, 1)

In [14]:
#df = pd.read_csv("../processed_data/4_18.csv", index_col=None)

In [101]:
df.head(2)

,male,title_Mr,title_Mrs,title_crew,title_highso,pclass_2,pclass_3,cabin_B,cabin_C,cabin_D,...,embarked_S,Family_Single,Family_Small,Family_Large,Age,Fare,Cabin,FamilySize,Ticket,Survived
0,1,1,0,0,0,0,1,0,0,0,...,1,0,1,0,0.273456,0.1,0.00,0.285714,0.014151,0.0
1,0,0,1,0,0,0,0,0,1,0,...,0,0,1,0,0.473882,0.1,0.25,0.285714,0.139136,1.0


In [102]:
df[891:].head(6)

,male,title_Mr,title_Mrs,title_crew,title_highso,pclass_2,pclass_3,cabin_B,cabin_C,cabin_D,...,embarked_S,Family_Single,Family_Small,Family_Large,Age,Fare,Cabin,FamilySize,Ticket,Survived
891,1,1,0,0,0,0,1,0,0,0,...,0,1,0,0,0.430039,0.0,0.0,0.0,0.015282,NaN
892,0,0,1,0,0,0,1,0,0,0,...,1,0,1,0,0.586622,0.1,0.0,0.0,0.013663,NaN
893,1,1,0,0,0,1,0,0,0,0,...,0,1,0,0,0.774521,0.0,0.0,0.0,0.018909,NaN
894,1,1,0,0,0,0,1,0,0,0,...,1,1,0,0,0.336089,0.0,0.0,0.0,0.016908,NaN
895,0,0,1,0,0,0,1,0,0,0,...,1,0,1,0,0.273456,0.2,0.0,0.0,0.023984,NaN
896,1,1,0,0,0,0,1,0,0,0,...,1,1,0,0,0.173243,0.0,0.0,0.0,0.018006,NaN


In [124]:
X = df.drop(['Survived'], axis = 1)

In [125]:
y = df['Survived'][:891]

In [126]:
y.shape

(891,)

In [127]:
X_train_v = X[0:891]
y_train_v = y
X_test_f = X[891:]

In [128]:
X_test_f = X_test_f.reset_index(drop=True)

In [129]:
X_test_f.shape

(418, 25)

In [130]:
print(X_train_v.shape)
np.linalg.matrix_rank(X_train_v) # needs to be full rank

(891, 25)


25

In [131]:
X_train, X_test, y_train, y_test = train_test_split(X_train_v, y_train_v,
                                                    test_size = 0.2)

#### General Linear

In [132]:
from sklearn.linear_model import LogisticRegression
logmodel = LogisticRegression()
logmodel.fit(X_train,y_train)
y_pred = logmodel.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))

[[102  12]
 [ 12  53]]
             precision    recall  f1-score   support

        0.0       0.89      0.89      0.89       114
        1.0       0.82      0.82      0.82        65

avg / total       0.87      0.87      0.87       179



#### Naive Bayes

In [133]:
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))

[[79 35]
 [ 7 58]]
             precision    recall  f1-score   support

        0.0       0.92      0.69      0.79       114
        1.0       0.62      0.89      0.73        65

avg / total       0.81      0.77      0.77       179



#### K nearest neighbor

In [134]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
classifier.fit(X_train, y_train)
# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))

[[96 18]
 [13 52]]
             precision    recall  f1-score   support

        0.0       0.88      0.84      0.86       114
        1.0       0.74      0.80      0.77        65

avg / total       0.83      0.83      0.83       179



#### Kernal SVM

In [135]:
from sklearn.svm import SVC
classifier = SVC(kernel = 'rbf', random_state = 0)
classifier.fit(X_train, y_train)

# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))

[[103  11]
 [ 15  50]]
             precision    recall  f1-score   support

        0.0       0.87      0.90      0.89       114
        1.0       0.82      0.77      0.79        65

avg / total       0.85      0.85      0.85       179



In [136]:
from keras.models import load_model

In [137]:
model = load_model('my_model.h5')

In [377]:
#kernel_regularizer=regularizers.l2(0.15))
#model.add(BatchNormalization())


In [31]:
K.clear_session()
model = Sequential()
model.add(Dense(16, input_shape=(25,), activation='relu', 
                kernel_initializer=initializers.glorot_normal(seed=123)))
model.add(Dropout(0.2))

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(16, activation='relu'))
model.add(Dropout(0.2))


model.add(Dense(8, activation='relu'))
model.add(Dropout(0.2))


model.add(Dense(4, activation='relu'))

model.add(Dense(1, activation='sigmoid'))
model.compile('Adam', 'binary_crossentropy', metrics=['accuracy'])

In [32]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 16)                416       
_________________________________________________________________
dropout_1 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               4352      
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 64)                16448     
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 16)                1040      
__________

In [40]:
model.fit(X_train, y_train, batch_size = 32,
          epochs=100, validation_split=0.3, verbose=0)


In [138]:
model.evaluate(X_test, y_test)

179/179 [==============================] - 0s 633us/step


[0.45129093744235332, 0.87709497040210493]

## STOP!!

Your saved model is 87%. Don't save over it.

In [449]:
model.save('my_model.h5')  
# creates a HDF5 file 'my_model.h5' has 87.7% accuracy

In [28]:
del model  # deletes the existing model

In [451]:
# returns a compiled model
# identical to the previous one
model = load_model('my_model.h5')

In [452]:
model.evaluate(X_test, y_test)

179/179 [==============================] - 0s 459us/step


[0.45129093744235332, 0.87709497040210493]

In [139]:
y_pred = model.predict(X_test)

In [140]:
y_pred_class = [ ]
for i in y_pred:
    if i >= 0.5:
        y_pred_class.append(1)
    else:
        y_pred_class.append(0)
        

In [141]:
y_test_class = y_test.values

In [142]:
confusion_matrix(y_test_class, y_pred_class)

array([[109,   5],
       [ 17,  48]])

In [143]:
print(classification_report(y_test_class, y_pred_class))

             precision    recall  f1-score   support

        0.0       0.87      0.96      0.91       114
        1.0       0.91      0.74      0.81        65

avg / total       0.88      0.88      0.87       179



Unfortunately, you'll need to preprocess all the test data step by step

In [ ]:
# passenger_id = full[891:].PassengerId 
# data set X doesn't have passenger id; I may need to add 
# that back on prior to exporting to csv

In [ ]:
Pass = pd.DataFrame(test.PassengerId)

In [144]:
y_final = [ ]
for i in y_pred:
    if i >= 0.5:
        y_final.append(1)
    else:
        y_final.append(0)
        

In [ ]:
test = pd.DataFrame({'PassengerId': passenger_id, 'Survived': y_final})


In [ ]:
test.head()

In [ ]:
test.to_csv('predictionANN_jander.csv', index=False)

### submit

In [146]:
X_test_f.head(1)

,male,title_Mr,title_Mrs,title_crew,title_highso,pclass_2,pclass_3,cabin_B,cabin_C,cabin_D,...,embarked_Q,embarked_S,Family_Single,Family_Small,Family_Large,Age,Fare,Cabin,FamilySize,Ticket
0,1,1,0,0,0,0,1,0,0,0,...,1,0,1,0,0,0.430039,0.0,0.0,0.0,0.015282


In [147]:
y_pred = model.predict(X_test_f)

In [148]:
y_final = [ ]
for i in y_pred:
    if i >= 0.5:
        y_final.append(1)
    else:
        y_final.append(0)
        

In [171]:
y_final = pd.DataFrame(y_final)


In [172]:
y_final.columns = ['Survived']

In [173]:
y_final.head()

,Survived
0,0
1,0
2,0
3,0
4,1


In [174]:
sub1 = pd.concat([Pass, y_final], axis = 1)

In [175]:
sub1.head(3)

,PassengerId,Survived
0,892,0
1,893,0
2,894,0


In [176]:
sub1.to_csv('predictionANN_jander.csv', index=False)